## 使用gluon 进行冰川识别
kaggle 冰川比赛[链接](https://www.kaggle.com/c/statoil-iceberg-classifier-challenge)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from subprocess import check_output
from matplotlib import pyplot as plt

%matplotlib inline

## 加载数据，并显示数据

In [ ]:
data_path = 'D:/dataset/kaggle_Iceberg/'
train_path = data_path + "train.json"
test_path  = data_path + "test.json"
train_json = pd.read_json(train_path)
test_json = pd.read_json(test_path)
train_json['inc_angle'] = pd.to_numeric(train_json['inc_angle'], errors='coerce')
test_json['inc_angle'] = pd.to_numeric(test_json['inc_angle'], errors='coerce')
print (len(train_json))

In [ ]:
print("Train Records")
train_json.head()


In [ ]:
print("Test Records")
test_json.head()

## 数据增强部分
对训练数据进行随机裁剪和镜像

In [ ]:
from mxnet import autograd
from mxnet import gluon
from mxnet import image
from mxnet import init
from mxnet import nd
from mxnet.gluon.data import vision

def data_norm(data):
    images = []
    for i,row in data.iterrows():
        band_1 = np.reshape(np.array(row["band_1"]).astype(np.float32),(75,75))
        band_2 = np.reshape(np.array(row["band_2"]).astype(np.float32),(75,75))
        band_3 = band_1+band_2

        band_1_norm = (band_1-band_1.mean())/ (band_1.max() - band_1.min())
        band_2_norm = (band_2-band_2.mean() ) / (band_2.max() - band_2.min())
        band_3_norm = (band_3 -band_3.mean()) / (band_3.max() - band_3.min())
        images.append(np.stack((band_1_norm,band_2_norm,band_3_norm)))
    return np.array(images)

def transform_train(data):
    data = nd.array(data)
    #data = (data-data.min())/(data.max()-data.min())
    auglist = image.CreateAugmenter(data_shape=(3, 75, 75), resize=0, 
                        rand_crop=True, rand_resize=True, rand_mirror=True,
                        brightness=0, contrast=0, 
                        saturation=0, hue=0, 
                        pca_noise=0, rand_gray=0, inter_method=2)
    
    for i in range(data.shape[0]):
        im = data[i,:]
        for aug in auglist:
            data[i,:] = aug(data[i,:])
    return data.asnumpy()

def transform_test(data):
    data = nd.array(data)
    auglist = image.CreateAugmenter(data_shape=(3, 75, 75))
    for i in range(data.shape[0]):
        for aug in auglist:
            data[i,:] = aug(data[i,:])
    return data.asnumpy()

def aug_img (data,aug_method):
    dataset = data_norm(data)
    dataset = np.transpose(dataset, (0,3,2,1))
    if aug_method == "train":
        dataset = transform_train(dataset) 
    elif aug_method == "test":
        dataset = transform_test(dataset)
    dataset = np.transpose(dataset, (0,3,2,1))
    return dataset

In [ ]:
X_train_total = aug_img(train_json,"train")
y_train_total = np.array(train_json['is_iceberg'])
X_angle_train_total = np.array(train_json.inc_angle)

X_test = aug_img(test_json,"test")
X_angle_test = np.array(test_json.inc_angle)

In [ ]:
print X_train_total.shape
# X_hah = data_norm(train_json)
# print (X_hah.max(),X_hah.min())
# img = X_train_total[10,:]
# img = np.transpose(img,(2,1,0))
# print img.max(),img.min()
# fig = plt.figure(1,figsize=(15,15))
# ax = fig.add_subplot(1,1,1)
# ax.imshow(img)

print (X_train_total.min(),X_train_total.max())

## 验证集

In [ ]:
X_train, X_valid, X_angle_train, X_angle_valid, y_train, y_valid = train_test_split(X_train_total
                    , X_angle_train_total, y_train_total, random_state=123, test_size=0.1)

    

y_test = np.zeros((X_test.shape[0],), dtype=np.int)

train_ds = gluon.data.ArrayDataset(X_train, y_train)
valid_ds = gluon.data.ArrayDataset(X_valid, y_valid)
train_valid_ds = gluon.data.ArrayDataset(X_train_total,y_train_total)
test_ds = gluon.data.ArrayDataset(X_test,y_test)

In [ ]:
print X_train.shape

In [ ]:
batch_size = 128
loader = gluon.data.DataLoader
train_data = loader(train_ds, batch_size, shuffle=True, last_batch='keep')
valid_data = loader(valid_ds, batch_size, shuffle=True, last_batch='keep')
train_valid_data= loader(train_valid_ds, batch_size, shuffle=True, last_batch='keep')
test_data = loader(test_ds,batch_size,shuffle = True,last_batch='keep')
# 交叉熵损失函数。
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

### 演示数据在块之间的形状变化
采用不同的net的形状变化 在这里进行显示。

In [ ]:
from mynet import * 
from vggnet import *

net = VggNet(2, verbose=True)
net.initialize()

x = nd.random.uniform(shape=(4, 3, 75, 75))
y = net(x)

## 训练

In [ ]:
import datetime
import sys
sys.path.append('..')
import utils
from matplotlib import pyplot as plt

def get_loss(data, net, ctx):
    loss = 0.0
    for feas, label in data:
        label = label.as_in_context(ctx)
        output = net(feas.as_in_context(ctx))
        cross_entropy = softmax_cross_entropy(output, label)
        loss += nd.mean(cross_entropy).asscalar()
    return loss / len(data)

def train(net, train_data, valid_data, num_epochs, lr, wd, ctx, lr_period,
          lr_decay):
    loss = 10
    trainer = gluon.Trainer(
        net.collect_params(), 'sgd', {'learning_rate': lr, 'momentum': 0.9,
                                      'wd': wd})
    prev_time = datetime.datetime.now()
    plt_train_loss = []
    plt_valid_loss = []

    for epoch in range(num_epochs):
        train_loss = 0.0
        if epoch < 81 and epoch % lr_period == 0:
            trainer.set_learning_rate(trainer.learning_rate * lr_decay)
        if epoch > 81 and epoch % 10 == 0:
            trainer.set_learning_rate(trainer.learning_rate * 0.1)
        for data, label in train_data:
            label = label.as_in_context(ctx)
            with autograd.record():
                output = net(data.as_in_context(ctx))
                loss = softmax_cross_entropy(output, label)
            loss.backward()
            trainer.step(batch_size)
            train_loss += nd.mean(loss).asscalar()
        cur_time = datetime.datetime.now()
        h, remainder = divmod((cur_time - prev_time).seconds, 3600)
        m, s = divmod(remainder, 60)
        time_str = "Time %02d:%02d:%02d" % (h, m, s)
        if valid_data is not None:
            valid_loss = get_loss(valid_data, net, ctx)
            epoch_str = ("Epoch %d. Train loss: %f, Valid loss %f, "
                         % (epoch, train_loss / len(train_data), valid_loss))
            plt_train_loss.append(train_loss / len(train_data))
            plt_valid_loss.append(valid_loss)
            
        else:
            epoch_str = ("Epoch %d. Train loss: %f, "
                         % (epoch, train_loss / len(train_data)))
        prev_time = cur_time
        if epoch % 10==0:
            print(epoch_str + time_str + ', lr ' + str(trainer.learning_rate))

    # plot 
    if valid_data is not None:
        plt.plot(plt_train_loss)
        plt.plot(plt_valid_loss)
        plt.legend(['train_loss','test_loss'])
        plt.savefig("Loss22.png")


In [ ]:
ctx = utils.try_gpu()
num_epochs = 140
learning_rate = 0.01
weight_decay = 5e-4
lr_period = 100
lr_decay = 0.1

# net = get_simple_net(ctx)
#net = get_simple_net(ctx,"resnet")
net  = vgg_net(ctx)
net.hybridize()
train(net, train_data, valid_data, num_epochs, learning_rate,weight_decay, ctx, lr_period, lr_decay)

In [ ]:
import numpy as np

net = get_simple_net(ctx)
net.hybridize()
train(net, train_valid_data, None, num_epochs, learning_rate, weight_decay,
      ctx, lr_period, lr_decay)

### 使用训练好的模型对测试样本进行预测，并保存文本

In [ ]:
outputs = []
for data, label in test_data:
    output = nd.softmax(net(data.as_in_context(ctx)))
    outputs.extend(output.asnumpy())

In [ ]:
test_pre = []

for num in outputs:
    test_pre.append(num[1])
print (len(test_pre)==8424)

In [ ]:
submission = pd.DataFrame({'id': test_json["id"], 'is_iceberg': test_pre})
submission.to_csv("submission.csv", index=False)